# Implementing Advanced Parallel Patterns with CUB
In previous notebooks, we explored how to directly write CUDA kernels using CuPy's `RawModule` to implement basic parallel processing patterns like map and stencil.
Specifically, in the last notebook, we delved into Shared Memory for cooperative processing within GPU blocks and Warp-level data exchange leveraging shuffle instructions.
These concrete examples of Warp and block-level implementations laid crucial groundwork for understanding CUB (CUDA UnBound).

This notebook introduces CUB, a high-performance library provided by NVIDIA, to tackle more challenging parallelization patterns such as reduction and scan.
CUB offers a rich set of functionalities to implement frequently used GPU programming patterns with high efficiency and versatility.

As a specific example for the reduction pattern, we'll focus on vector normalization.
Through this process, we'll demonstrate how to use CUB's `BlockReduce` to efficiently compute the norm in parallel.
Subsequently, we'll use prefix sum as our subject to explain the scan pattern, implementing it with CUB's `BlockScan`.

By understanding these advanced parallel patterns and the importance of Coalesced Access to memory, you'll gain a foothold for optimizing more complex GPU computations and significantly boosting application performance.

In [ ]:
import os
import math
import numpy as np
if os.environ.get('NVCC') is None:
  os.environ['NVCC'] = '/usr/local/cuda/bin/nvcc'
import cupy as cp

err_eps = 1E-7
block_size = 1024
items_per_thread = 8
length = block_size * items_per_thread

dn = os.path.join(os.getcwd(), 'kernels')
fpfn = os.path.join(dn, '04_advanced_parallel_patterns_1.cu')
with open(fpfn, 'r') as f:
  cuda_source = f.read()
print(cuda_source)

The `vectorNormalization` kernel calculates the vector normalization in parallel.

First, it uses `BlockLoad` to read data from global memory.
The `cub::BLOCK_LOAD_STRIPED` pattern used here is a technique to maximize GPU memory efficiency.
GPUs can achieve higher speeds by reading and writing multiple threads to contiguous memory addresses simultaneously, a process known as Coalesced Access.
In `cub::BLOCK_LOAD_STRIPED`, each thread within a thread block reads elements from global memory at regular intervals, such as `32` elements (Warp size).
For example, thread `0` reads elements `0`, `32`, `64`, and so on, while thread `1` reads elements `1`, `33`, `65`, and so on.
The `threadData` array held by each thread stores these interleaved elements sequentially.
For computations where the order of individual elements doesn't affect the result, like the sum of squares calculation in this kernel, the performance benefits of Coalesced Access using the `STRIPED` pattern are significant, making it a suitable choice.
For more details on data load patterns, refer to [the Flexible Data Arrangement section of the CUB official documentation](https://nvidia.github.io/cccl/cub/index.html#flexible-data-arrangement).
Similarly, when writing computed results back to global memory, `cub::BLOCK_STORE_STRIPED` ensures efficient Coalesced Access for writes.

CUB primitives such as `BlockLoad`, `BlockReduce`, and the subsequent `BlockShuffle` and `BlockStore` internally utilize fast Shared Memory as a temporary workspace.
In this kernel, to efficiently manage the temporary storage used by these primitives, we place them in the same physical memory space using `__shared__ union { ... } tempStorage;`.
This is a practically important optimization for efficiently reusing the limited GPU Shared Memory and enhancing kernel execution efficiency.

Next, we calculate the sum of squares of the vector.
This is a crucial parallel pattern called reduction, which aggregates many elements into a single result.
Each thread handles a number of elements specified by `ITEMS_PER_THREAD`, individually squaring them and storing them in `threadDataSqr`.
These squared values are then efficiently aggregated by `cub::BlockReduce` to compute the sum of squares, `sqrSum`.
Such an implementation is possible because the order of individual elements does not affect the final sum of squares.

`__syncthreads();` is critical for inter-thread cooperative computations like reduction.
This synchronization is essential to ensure that all threads within the block have completed squaring their assigned elements before `BlockReduce` begins its sum of squares calculation.
Without data fully prepared, accurate results cannot be obtained.
This synchronization also ensures that `BlockLoad` and `BlockReduce` can safely reuse the same Shared Memory region via the union.
While CUB primitives perform necessary internal synchronization, explicit synchronization by the developer is crucial when sharing Shared Memory between different primitives and guaranteeing the order of operations and data readiness.

Subsequently, we compute the norm using the calculated sum of squares and then normalize each element.
The sum of squares, which is the result of `BlockReduce`, holds a valid value only for the thread with `threadIdx.x == 0` within the thread block (the leader thread).
To normalize elements per thread, the norm value is needed by all threads in the block.
Therefore, we use `cub::BlockShuffle` to broadcast this value.
We perform another `__syncthreads();` here to ensure that all normalization calculations are complete.
In broadcasting using `BlockShuffle`, we pass an array `normDummy[1]` as an argument.
This is because the `cub::BlockShuffle::Down` method is designed to receive the source value for broadcasting as a reference to an array.

Finally, we use `BlockStore` to write the normalized vector.

In [ ]:
cuda_source = cuda_source.replace('ADVANCED_PARALLEL_PATTERNS_1_BLOCK_SIZE', str(block_size))
cuda_source = cuda_source.replace('ADVANCED_PARALLEL_PATTERNS_1_ITEMS_PER_THREAD', str(items_per_thread))
module = cp.RawModule(code=cuda_source, backend='nvcc')
module.compile()

For compiling this kernel, we specify `backend='nvcc'`.
This is necessary because we are using official NVIDIA CUDA libraries like CUB.
As explained previously, CuPy's default compiler may not support these, so we explicitly use `nvcc`.

Finally, we execute the GPU kernel using CuPy and verify the results by comparing them with a NumPy calculation.
If the computation is successful, no assertion error will occur.

In [ ]:
x = np.random.rand((length)).astype(np.float32)
x_gpu = cp.array(x, dtype=cp.float32)
y_gpu = cp.empty_like(x)
assert x_gpu.flags.c_contiguous
assert y_gpu.flags.c_contiguous

In [ ]:
gpu_func = module.get_function('vectorNormalization')
sz_block = block_size,
sz_grid = 1,
gpu_func(
  block=sz_block, grid=sz_grid,
  args=(y_gpu, x_gpu)
)
cp.cuda.runtime.deviceSynchronize()

In [ ]:
y = y_gpu.get()
y_ref = x / np.linalg.norm(x)

err = np.abs(y_ref - y)
assert np.max(err) < err_eps

Thus far, we've explained three important elements of GPU parallel processing, using vector normalization as an example.
Specifically, we've looked at the reduction pattern (sum of squares), broadcasting values using `BlockShuffle`, and the crucial `STRIPED` reads/writes and Coalesced Access for performance.

Next, we'll explain a different parallel pattern: prefix sum (scan pattern). This scan has different characteristics and implementation considerations compared to reduction.

First, let's read the kernel.

In [ ]:
dn = os.path.join(os.getcwd(), 'kernels')
fpfn = os.path.join(dn, '04_advanced_parallel_patterns_2.cu')
with open(fpfn, 'r') as f:
  cuda_source = f.read()
print(cuda_source)

Let's look at `parallelPrefixSum`, which implements prefix sum (cumulative sum).
This operation calculates the total value up to each element in a given sequence.
For example, the prefix sum of `[1, 2, 3, 4]` is `[1, 3, 6, 10]`.
This is called the scan pattern in the context of parallel algorithms and is generally difficult to parallelize because each output element depends on all preceding input elements.

This kernel also uses `BlockLoad` and `BlockStore` to read and write data.
However, unlike the previous normalization kernel, it's crucial to note that here we specify the `cub::BLOCK_LOAD_DIRECT` and `cub::BLOCK_STORE_DIRECT` patterns.
Since the order of elements directly affects the result in prefix sum, the data arrangement held by each thread must be contiguous in global memory.
In the `DIRECT` pattern, the `ITEMS_PER_THREAD` elements handled by each thread read and write a contiguous range in global memory (e.g., thread `0` reads from element `0` to `ITEMS_PER_THREAD-1`, thread `1` reads from `ITEMS_PER_THREAD` to `2*ITEMS_PER_THREAD-1`, and so on).

Next, we'll compile and execute this kernel, then verify its results.
This time, we've changed the element type to `int32`.
For verification, we'll use NumPy's `numpy.cumsum()` to calculate the reference cumulative sum and compare it with the GPU result.
If the computation is successful, no assertion will occur.

In [ ]:
cuda_source = cuda_source.replace('ADVANCED_PARALLEL_PATTERNS_2_BLOCK_SIZE', str(block_size))
cuda_source = cuda_source.replace('ADVANCED_PARALLEL_PATTERNS_2_ITEMS_PER_THREAD', str(items_per_thread))
module = cp.RawModule(code=cuda_source, backend='nvcc')
module.compile()

In [ ]:
x = (1024 * np.random.rand((length))).astype(np.int32)
x_gpu = cp.array(x, dtype=cp.int32)
y_gpu = cp.empty_like(x)
assert x_gpu.flags.c_contiguous
assert y_gpu.flags.c_contiguous

In [ ]:
gpu_func = module.get_function('parallelPrefixSum')
sz_block = block_size,
sz_grid = 1,
gpu_func(
  block=sz_block, grid=sz_grid,
  args=(y_gpu, x_gpu)
)
cp.cuda.runtime.deviceSynchronize()

In [ ]:
y = y_gpu.get()
y_ref = np.cumsum(x)

err = np.abs(y_ref - y)
assert np.max(err) < err_eps

In this notebook, we covered two advanced methods for implementing parallel patterns in CUDA: vector normalization as an example of reduction, and prefix sum (cumulative sum) as an example of scan.
For both patterns, we demonstrated through fundamental and concrete code how powerful CUB is in maximizing GPU performance.

First, for vector normalization, we explained the flow from independent element calculation and reduction (sum of squares) to broadcasting the result.
Notably, we implemented Coalesced Access, crucial for achieving GPU memory efficiency, using patterns like `cub::BLOCK_LOAD_STRIPED` and `cub::BLOCK_STORE_STRIPED`.

Next, for prefix sum, we saw how `cub::BlockScan` can concisely describe the complex logic inherent in the scan pattern, which has a particular difficulty in parallelization due to each element depending on its predecessors.
Here, we also showed that a different load pattern, `cub::BLOCK_LOAD_DIRECT`, is more suitable because data order is critical.

Common to both kernels was the importance of using a union struct to efficiently reuse the limited Shared Memory resource, and the necessity of explicit `__syncthreads()` for guaranteeing computational correctness.
While CUB encapsulates complex low-level synchronization, developers must manage synchronization timing themselves when ensuring that data for CUB operations is fully computed or when reusing Shared Memory between different CUB primitives.

Thus, we've shown how complex parallel patterns can be implemented efficiently and concisely by leveraging CUB.